In [1]:
# import libraries
import pandas as pd
import numpy as np 
import warnings
import gmplot
import sys
import geopy
from geopy import distance
from copy import deepcopy
sys.path.append("../HistoricalData/")
from getData import get_data

warnings.filterwarnings('ignore')

In [12]:
# this call to get_data function that will take bounding box and timeframe and return cleaned data

UP_LEFT = (38.008050, -122.536985)    # (lat, lon)
UP_RIGHT = (38.008050, -122.186437)   # (lat, lon)
DOWN_RIGHT = (37.701933, -122.186437) # (lat, lon)
DOWN_LEFT = (37.701933, -122.536985)  # (lat, lon)
START_DATE = '2019/09/10' # begin date to start taking data
END_DATE = '2019/09/15'   # end date to start taking data
START_HOUR = '0'         # hour EACH DAY to start, this allows us to control for time of day effects
END_HOUR = '24'           # hour EACH DAY to end, this allows us to control for time of day effects

data_df = get_data(UP_LEFT, UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, START_DATE, END_DATE, START_HOUR, END_HOUR)

In [16]:
class Chain:
    """ 
    Chain is a class that defines a chain of sensors in time and space. 
    A chain has a list of timestamps that are paired with a list of particle locations, intersecting sensors, and readings
    """
    def __init__(self, sensor_id, start_time):
        self.start_time = start_time
        self.time = [start_time]
        self.sensor_path = [sensor_id]
        self.last_sensor = sensor_id
        self.particle_path = [(data_df[data_df.sensor_id == sensor_id].lat.iloc[0], data_df[data_df.sensor_id == sensor_id].lon.iloc[0])]
        try:
            self.readings = [data_df[(data_df.sensor_id == sensor_id) & (data_df.created == start_time)]['2_5um'].iloc[0]]
        except:
            self.readings = -999 # drop this when found on a chain
    
    def plot_next(self):
        # update timed
        MINUTES = 10 # Joined data seems to be in increments of 10 minutes
        origin_time = self.time[-1]
        destination_time = self.time[-1] + MINUTES
        self.time.append(destination_time)
        
        # need to take wind speed in knots
        origin = geopy.Point(self.particle_path[-1][0], self.particle_path[-1][1])
        km_1_knot_per_hour = 1.852
        km_hour_test = km_1_knot_per_hour * data_df[(data_df.sensor_id == self.last_sensor) & (data_df.created == origin_time)].wind_speed
        if len(km_hour_test) == 0: # handle empty data
            return False
        km_hour = km_hour_test.iloc[0] # so you are grabbing from last sensor seen... might not be nearest wind data
        d = (MINUTES / 60) * km_hour 
    
        # convert wind_direction to a bearing
        bearing = data_df[(data_df.sensor_id == self.last_sensor) & (data_df.created == origin_time)].wind_direction.iloc[0] 
        try:
            b = int(bearing) + 180
        except: # 'VRB' = variable winds
            return False 
        if b >= 360:
            b -= 360
        
        # given: lat1, lon1, b = bearing in degrees, d = distance in kilometers
        if d == 0:
            destination = self.particle_path[-1]
        else:
            destination = geopy.distance.VincentyDistance(kilometers=d).destination(origin, b)
            destination = (destination.latitude, destination.longitude)
        
            # end chain if outside bounding box
            if (destination[0] > UP_LEFT[0]) or (destination[0] < DOWN_LEFT[0]) or (destination[1] < UP_LEFT[1]) or (destination[1] > DOWN_RIGHT[1]):
                return False
        
            self.particle_path.append(destination)
        
            # check is there's a sensor nearby
            CLOSE_ENOUGH_DISTANCE = 1 # have to be within 1000 meters
        
            closest_sensor_id = None
            closest_sensor_dist = CLOSE_ENOUGH_DISTANCE
            
            # get data from a much tighter bounding box
            center = geopy.point.Point(destination)
            km_away = geopy.distance.VincentyDistance(kilometers = max(CLOSE_ENOUGH_DISTANCE,d))
            north = geopy.point.Point(km_away.destination(point=center, bearing=0))
            east = geopy.point.Point(km_away.destination(point=center, bearing=90))
            south = geopy.point.Point(km_away.destination(point=center, bearing=180))
            west = geopy.point.Point(km_away.destination(point=center, bearing=270))

            nearby_df = data_df[(data_df.created == destination_time) & (data_df.lat < north.latitude) & (data_df.lat > south.latitude) & (data_df.lon < east.longitude) & (data_df.lon > west.longitude)]

            for sensor in nearby_df.sensor_id.unique(): # iterate through all sensors in bounding box
                test_sensor = (float(nearby_df[nearby_df.sensor_id == sensor].lat.iloc[0]), float(nearby_df[nearby_df.sensor_id == sensor].lon.iloc[0]))
                testy = geopy.point.Point(test_sensor)
                desty = geopy.point.Point(destination)
                proximity = geopy.distance.distance(testy, desty).km
                if (proximity < closest_sensor_dist):
                    closest_sensor_id = sensor
                    closest_sensor_dist = proximity
                
            # update other lists
            self.sensor_path.append(closest_sensor_id)
            if closest_sensor_id:
                self.last_sensor = closest_sensor_id
                try: # handle empty reading
                    self.readings.append(data_df[(data_df.sensor_id == closest_sensor_id) & (data_df.created == destination_time)]['2_5um'].iloc[0])
                    if self.readings[0] == -999: # handle badly formed chain by removing all links but the most recent sensor
                        self.readings = self.readings[-1]
                        self.sensor_path = self.sensor_path[-1]
                        self.particle_path = self.particle_path[-1]
                except:
                    return False
        return True
    
    def plot_pollution(self):
        THRESHOLD = 3 # how much of a pm2.5 count increase needs to happen to highlight a link
        lats = []
        lons = []
        colors= [0.1] # first color not used. had been white
        link = 0 # counts the links
        reading = 0 # counts the readings
        for point in self.particle_path:
            lats.append(point[0])
            lons.append(point[1])
            if (link > 0) and (self.sensor_path[link]):
                reading += 1
                if reading > (len(self.readings)-1): # out of index, missing reading:
                    colors.append(0.1) # had been black
                elif (self.readings[reading] > (self.readings[reading-1] + THRESHOLD)): # possible pollution source
                    colors.append(1) # had been red
                    look_back = link-1
                    while ((look_back >= 0) and (self.sensor_path[look_back] == None)):
                        colors.append(1) # had been red
                        look_back -= 1
                else: # no pollution source
                    colors.append(0.1) # had been blue
                    look_back = link-1
                    while ((look_back >= 0) and (self.sensor_path[look_back] == None)):
                        colors.append(0.1) # had been blue
                        look_back -= 1
            link += 1

        # fill in black color for chain links at end that never feed into another sensor
        if len(self.particle_path) > 1:
            for index in range(len(self.particle_path),-1,-1):
                try:
                    if self.sensor_path[index] == None:
                        colors.append(0.1) # had been black
                    else:
                        return lats, lons, colors
                except:
                    colors.append(0.1) # had been black
        return lats, lons, colors

In [17]:
chains = [] # a list of chain 

set_size_chain = 10
start_time = 201909101000 # noon?

# how many time cycles?
cycles = 5
for clock in range(start_time, start_time + 10 * cycles, 10):
    print("Clock", clock)
    # build the chains
    for sensor in data_df.sensor_id.unique(): # iterate through all sensors in bounding box
        chain = Chain(sensor, clock) # init one chain per sensor
        time_step = 0
        while (chain.plot_next() & (time_step < set_size_chain)): # method returns false if out of bounds/data
            time_step += 1
        chains.append(chain) # add the chain object to the list of chains
        print("Chain", len(chains), "is", chain.sensor_path, "with readings", chain.readings)
    print("Done.")

Clock 201909101000
Chain 1 is ['16939', '16943', None, None] with readings [13.21, 14.11]
Chain 2 is ['16940', '16943', None, None] with readings [6.25, 14.11]
Chain 3 is ['16919', '16943', None, None] with readings [10.29, 14.11]
Chain 4 is ['16920', '16943', None, None] with readings [7.06, 14.11]
Chain 5 is ['16931', '16943', None, None] with readings [1.5, 14.11]
Chain 6 is ['16932', '16943', None, None] with readings [1.38, 14.11]
Chain 7 is ['19173', '16943', None, None] with readings [5.04, 14.11]
Chain 8 is ['19174', '16943', None, None] with readings [4.42, 14.11]
Chain 9 is ['16947', '16943', None, None] with readings [5.28, 14.11]
Chain 10 is ['16948', '16943', None, None] with readings [10.06, 14.11]
Chain 11 is ['18943', None, None, '29537', '19299', None] with readings [2.63, 3.31, 8.84]
Chain 12 is ['18944', None, None, '29537', '19299', None] with readings [2.22, 3.31, 8.84]
Chain 13 is ['20747'] with readings [2.93]
Chain 14 is ['20748'] with readings [7.0]
Chain 15 is

Chain 123 is ['26354', '21443', None, None, None, None] with readings [6.3, 15.94]
Chain 124 is ['16935', '16943', None, None] with readings [5.15, 14.11]
Chain 125 is ['16936', '16943', None, None] with readings [7.48, 14.11]
Chain 126 is ['17805', '16943', None, None] with readings [8.57, 14.11]
Chain 127 is ['17806', '16943', None, None] with readings [10.96, 14.11]
Chain 128 is ['6182', None, None, None, None] with readings [2.68]
Chain 129 is ['6183', None, None, None, None] with readings [3.55]
Chain 130 is ['19155', '8468', None, '21251'] with readings [8.44, 0.0, 8.41]
Chain 131 is ['19156', '8468', None, '21251'] with readings [2.12, 0.0, 8.41]
Chain 132 is ['19301', '25625'] with readings [12.75, 1.29]
Chain 133 is ['19302', '25625'] with readings [2.84, 1.29]
Chain 134 is ['19411', None, None, None, None] with readings [5.25]
Chain 135 is ['19412', None, None, None, None] with readings [11.82]
Chain 136 is ['20515', '7198', None, None, None, '20293', None] with readings [4.2

Chain 252 is ['3666', None, None, None, None] with readings [5.34]
Chain 253 is ['3667', None, None, None, None] with readings [6.38]
Chain 254 is ['11606', None] with readings [nan]
Chain 255 is ['11607', None] with readings [nan]
Chain 256 is ['11628', None] with readings [nan]
Chain 257 is ['11629', None] with readings [nan]
Chain 258 is ['17411', '19405', '21251', None] with readings [2.35, 1.86, 7.19]
Chain 259 is ['18287', '19405', '21251', None] with readings [3.96, 1.86, 7.19]
Chain 260 is ['18288', '19405', '21251', None] with readings [5.61, 1.86, 7.19]
Chain 261 is ['25011'] with readings [3.61]
Chain 262 is ['19261', '16943', None, None, None] with readings [2.38, 14.11]
Chain 263 is ['19262', '16943', None, None, None] with readings [1.7, 14.11]
Chain 264 is ['22051', '16943', None, None, None] with readings [4.66, 14.11]
Chain 265 is ['22052', '16943', None, None, None] with readings [4.39, 14.11]
Chain 266 is ['19849', '22117'] with readings [1.3, 1.5]
Chain 267 is ['230

Chain 370 is ['19713', None, '14811', None, None, None, None] with readings [0.76, 3.84]
Chain 371 is ['19714', None, '14811', None, None, None, None] with readings [3.74, 3.84]
Chain 372 is ['19039', None, None, None, None, '19693', None] with readings [2.86, 3.81]
Chain 373 is ['19040', None, None, None, None, '19693', None] with readings [2.38, 3.81]
Chain 374 is ['19813', None] with readings [1.51]
Chain 375 is ['26311', None, None, '5236', '5282', '19599', None] with readings [nan, 0.0, 3.41, 9.22]
Chain 376 is ['26312', None, None, '5236', '5282', '19599', None] with readings [nan, 0.0, 3.41, 9.22]
Chain 377 is ['19315', None, None, None, '19299', None] with readings [2.5, 8.84]
Chain 378 is ['19316', None, None, None, '19299', None] with readings [1.58, 8.84]
Chain 379 is ['22263', None, None, None, '19299', None] with readings [7.48, 8.84]
Chain 380 is ['20025', '24131', None, None] with readings [10.73, 7.45]
Chain 381 is ['4772', None, None] with readings [1.59]
Chain 382 is 

Chain 489 is ['19815', '18943', None, None, '5270', None, None] with readings [0.25, 2.9, 3.24]
Chain 490 is ['22177', '24223', None, None, '19299', None] with readings [9.74, 4.85, 8.84]
Chain 491 is ['15769', None] with readings [5.17]
Chain 492 is ['15770', None] with readings [3.28]
Chain 493 is ['30189', '25791', None, None] with readings [6.46, 2.57]
Chain 494 is ['30190', '25791', None, None] with readings [2.37, 2.57]
Chain 495 is ['17927', '19275', '25625'] with readings [9.01, 5.94, 1.42]
Chain 496 is ['17928', '19275', '25625'] with readings [3.54, 5.94, 1.42]
Chain 497 is ['10066', '35433', None, None, None] with readings [2.84, 11.46]
Chain 498 is ['10067', '35433', None, None, None] with readings [5.0, 11.46]
Chain 499 is ['19657', None, None, None, None] with readings [3.94]
Chain 500 is ['19658', None, None, None, None] with readings [5.09]
Chain 501 is ['3909', None, None, None] with readings [5.63]
Chain 502 is ['3910', None, None, None] with readings [9.29]
Chain 503

...

Chain 2002 is ['20000', '27257', None] with readings [6.5, 2.09]
Chain 2003 is ['18903', None, None] with readings [10.91]
Chain 2004 is ['18904', None, None] with readings [4.45]
Chain 2005 is ['2031'] with readings -999
Chain 2006 is ['2032'] with readings -999
Chain 2007 is ['28261', '23965', '13517', None] with readings [6.75, 9.07, 2.29]
Chain 2008 is ['19287'] with readings [6.29]
Chain 2009 is ['19288'] with readings [5.03]
Chain 2010 is ['36035', '20179', None] with readings [6.99, 5.32]
Chain 2011 is ['36036', '20179', None] with readings [5.57, 5.32]
Chain 2012 is ['19189', None, None, None] with readings [8.75]
Chain 2013 is ['19190', None, None, None] with readings [7.12]
Chain 2014 is ['20501', None, None, None] with readings [10.03]
Chain 2015 is ['20502', None, None, None] with readings [3.65]
Chain 2016 is ['4421', None, '21251'] with readings [1.38, 7.58]
Chain 2017 is ['4422', None, '21251'] with readings [2.44, 7.58]
Chain 2018 is ['19811', None, '21251'] with readin

Chain 2140 is ['26065', None, None] with readings [7.65]
Chain 2141 is ['26066', None, None] with readings [9.4]
Chain 2142 is ['19857', None, None] with readings [6.03]
Chain 2143 is ['15465', None, None] with readings [0.38]
Chain 2144 is ['15466', None, None] with readings [0.67]
Chain 2145 is ['4770', None, None] with readings [2.85]
Chain 2146 is ['4771', None, None] with readings [4.26]
Chain 2147 is ['5141', None] with readings [0.57]
Chain 2148 is ['5142', None] with readings [0.2]
Chain 2149 is ['22351', None, None] with readings [4.97]
Chain 2150 is ['22352', None, None] with readings [3.57]
Chain 2151 is ['29537', '4801', None] with readings [0.91, 0.94]
Chain 2152 is ['19299', None] with readings [8.84]
Chain 2153 is ['19300', None] with readings [11.64]
Chain 2154 is ['19153', None, None] with readings [3.97]
Chain 2155 is ['19154', None, None] with readings [4.89]
Chain 2156 is ['19377', '14769', None] with readings [1.06, 19.5]
Chain 2157 is ['19378', '14769', None] with

Chain 2284 is ['19997', '16943', None] with readings [2.17, 15.86]
Chain 2285 is ['19998', '16943', None] with readings [1.32, 15.86]
Chain 2286 is ['18491', '28365', None] with readings [5.39, 3.06]
Chain 2287 is ['18492', '28365', None] with readings [11.44, 3.06]
Chain 2288 is ['27135', '20951'] with readings [9.67, 3.67]
Chain 2289 is ['27136', '20951'] with readings [3.88, 3.67]
Chain 2290 is ['3900', '19711', None] with readings [3.14, 3.12]
Chain 2291 is ['3901', '19711', None] with readings [3.6, 3.12]
Chain 2292 is ['33491', '25791', None] with readings [1.59, 1.97]
Chain 2293 is ['33492', '25791', None] with readings [5.52, 1.97]
Chain 2294 is ['20131', None, None] with readings [2.6]
Chain 2295 is ['20132', None, None] with readings [2.93]
Chain 2296 is ['20781', '19301', None] with readings [3.47, 8.82]
Chain 2297 is ['20782', '19301', None] with readings [4.41, 8.82]
Chain 2298 is ['19447', None] with readings [2.46]
Chain 2299 is ['19448', None] with readings [4.0]
Chain 

Chain 2425 is ['27359', '4506', None] with readings [11.95, 3.93]
Chain 2426 is ['27360', '4506', None] with readings [6.15, 3.93]
Chain 2427 is ['22825', None, None] with readings [0.86]
Chain 2428 is ['4339'] with readings [3.3]
Chain 2429 is ['4340'] with readings [4.22]
Chain 2430 is ['21251'] with readings [6.35]
Chain 2431 is ['21252'] with readings [4.25]
Chain 2432 is ['14339', None, None] with readings [2.96]
Chain 2433 is ['14340', None, None] with readings [3.87]
Chain 2434 is ['19159', '23957', None] with readings [7.09, 4.43]
Chain 2435 is ['19160', '23957', None] with readings [9.05, 4.43]
Chain 2436 is ['17787', '22873', None] with readings [7.71, 11.67]
Chain 2437 is ['17788', '22873', None] with readings [11.65, 11.67]
Chain 2438 is ['1743'] with readings -999
Chain 2439 is ['21235', None, None] with readings [1.77]
Chain 2440 is ['21236', None, None] with readings [2.09]
Chain 2441 is ['31237', '11600', None] with readings [4.17, 2.72]
Chain 2442 is ['31238', '11600',

Chain 2569 is ['18281', '24999', None] with readings [5.09, 6.59]
Chain 2570 is ['18282', '24999', None] with readings [3.13, 6.59]
Chain 2571 is ['22523', None, None] with readings [3.91]
Chain 2572 is ['22524', None, None] with readings [0.0]
Chain 2573 is ['20785', None, None] with readings [1.07]
Chain 2574 is ['20786', None, None] with readings [4.54]
Chain 2575 is ['20989', None, None] with readings [10.45]
Chain 2576 is ['20990', None, None] with readings [2.98]
Chain 2577 is ['35809', None, None] with readings [5.36]
Chain 2578 is ['35810', None, None] with readings [5.52]
Chain 2579 is ['30413', None, None] with readings [1.97]
Chain 2580 is ['30414', None, None] with readings [1.19]
Chain 2581 is ['19815', None, None] with readings [0.58]
Chain 2582 is ['22177', None, None] with readings [5.49]
Chain 2583 is ['15769', None, None] with readings [5.0]
Chain 2584 is ['15770', None, None] with readings [3.1]
Chain 2585 is ['30189', '26063', None] with readings [6.54, 0.99]
Chain 

In [18]:
# plot it
latitude_list = []
longitude_list = []
for sensor in data_df.sensor_id.unique():
    try:
        latitude_list.append(float(data_df[data_df.sensor_id == sensor].lat.iloc[0]))
        longitude_list.append(float(data_df[data_df.sensor_id == sensor].lon.iloc[0]))
    except:
        pass
gmap3 = gmplot.GoogleMapPlotter((max(latitude_list)+min(latitude_list))/2,
                                (max(longitude_list)+min(longitude_list))/2,
                                zoom=11)
gmap3.scatter(latitude_list, longitude_list, '#FF0000', 
                              size = 40, marker = False ) 

debug = 0
for chain in chains:
    debug +=1
    lats, lons, colors = chain.plot_pollution()
    for index in range(1,len(lats)):
        try:
            #gmap3.heatmap([lats[index-1], lats[index]], [lons[index-1], lons[index]], threshold=10, radius=40, dissipating=False)
            #gmap3.heatmap([lats[index-1], lats[index]], [lons[index-1], lons[index]])
            #gmap3.heatmap([lats[index-1], lats[index]], [lons[index-1], lons[index]], threshold=10, radius=10, gradient=None, opacity=0.6, maxIntensity=1, dissipating=True)
            #def heatmap(self, lats, lngs, threshold=10, radius=10, gradient=None, opacity=0.6, maxIntensity=1, dissipating=True):
            gmap3.plot([lats[index-1], lats[index]], [lons[index-1], lons[index]], color = 'red', edge_width=colors[index])
        except:
            break

          
# draw the map and send to file
gmap3.draw("../data/chain_map.html") 
print("Done")

Done


### specific map output parameters
chain_map_201909101000.html:  
START_DATE = '2019/09/10'   
END_DATE = '2019/09/15'      
START_HOUR = '0'          
END_HOUR = '24'   
set_size_chain = 10  
start_time = 201909101000 # noon?  
cycles = 5  
for clock in range(start_time, start_time + 10000 * cycles, 10000)  